# Preparing

### Import everything needed

In [ ]:
from collections import Counter
import csv
from itertools import groupby
import random

import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage as sndi

%matplotlib inline

### Define all global varaibles

In [ ]:
drive_log_file = 'driving_log.csv'

# raw data
X_center = []
X_left = []
X_right = []
y_steering = []

# processed data
X_train = np.array([]) # path to the image
X_flag = np.array([]) # flag for idicating the image need to be flipped or not
y_train = np.array([]) # steering angle while the image was captured

### Define some helper function

In [ ]:
def show_steering_angle_distribution(y_train, groups = 2000):
    plt.title("Steering angle distribution in training data")
    plt.hist(y_train, groups)
    plt.show()

# Read raw data

In [ ]:
with open(drive_log_file, 'rt') as csvfile:
    reader = csv.reader(csvfile, skipinitialspace=True)

    next(reader) # skip the first row
    
    i = 0
    for row in reader:
        i = i+1
        X_center.append(row[0])
        X_left.append(row[1])
        X_right.append(row[2])
        y_steering.append(float(row[3]))


X_center = np.array(X_center)
X_left = np.array(X_left)
X_right = np.array(X_right)
y_steering = np.array(y_steering)

# Exploring

In [ ]:
show_steering_angle_distribution(y_steering)

positive_angle = [i for i in y_steering if i > 0]
nagative_angle = [i for i in y_steering if i < 0]
zero_angle = [i for i in y_steering if i == 0]

print('number of positive angle:', len(positive_angle))
print('number of negative angle:', len(nagative_angle))
print('number of zero angle:', len(zero_angle))

# Preprocessing

## Use side camera images

In [ ]:
X_train = np.append(X_center, X_left, axis=0)
X_flag = np.append(np.zeros((len(X_center))), np.zeros((len(X_left))), axis=0)
y_train = np.append(y_steering, y_steering+0.15, axis=0)

X_train = np.append(X_train, X_right, axis=0)
X_flag = np.append(X_flag, np.zeros((len(X_right))), axis=0)
y_train = np.append(y_train, y_steering-0.15, axis=0)

In [ ]:
show_steering_angle_distribution(y_train)

positive_angle = [i for i in y_train if i > 0]
nagative_angle = [i for i in y_train if i < 0]
zero_angle = [i for i in y_train if i == 0]

print('number of positive angle:', len(positive_angle))
print('number of negative angle:', len(nagative_angle))
print('number of zero angle:', len(zero_angle))

## Filter some zeros

In [ ]:
# indices = np.where(y_train == 0)
# indices = np.append(indices, np.where(y_train == 0.15))
# indices = np.append(indices, np.where(y_train == -0.15))

# np.random.shuffle(indices)
# indices = indices[:round(len(indices)*0.5)]

# X_flag = np.delete(X_flag, indices)
# X_train = np.delete(X_train, indices)
# y_train = np.delete(y_train, indices)

In [ ]:
# show_steering_angle_distribution(y_train)

# positive_angle = [i for i in y_train if i > 0]
# nagative_angle = [i for i in y_train if i < 0]
# zero_angle = [i for i in y_train if i == 0]

# print('number of positive angle:', len(positive_angle))
# print('number of negative angle:', len(nagative_angle))
# print('number of zero angle:', len(zero_angle))

## 2. Flip the image horizontally

In [ ]:
X_flag = np.append(X_flag, np.ones((len(X_train))), axis=0) # X_flag should be appended before X_train
X_train = np.append(X_train, X_train, axis=0)
y_train = np.append(y_train, -y_train, axis=0)

In [ ]:
show_steering_angle_distribution(y_train)

positive_angle = [i for i in y_train if i > 0]
nagative_angle = [i for i in y_train if i < 0]
zero_angle = [i for i in y_train if i == 0]

print('number of positive angle:', len(positive_angle))
print('number of negative angle:', len(nagative_angle))
print('number of zero angle:', len(zero_angle))

## 3. Select some of the data for training

In [ ]:
max_keep = 500

In [ ]:
for i in np.arange(-1.5, 1.5, 0.001):
    rounded_y_train = np.round(y_train, 3)
    i = np.round(i, 3)
    indices = np.where(rounded_y_train == i)[0]
    if len(indices) > max_keep:
        indices_to_be_deleted = indices[random.sample(range(len(indices)), len(indices) - max_keep)]
        X_flag = np.delete(X_flag, indices_to_be_deleted)
        X_train = np.delete(X_train, indices_to_be_deleted)
        y_train = np.delete(y_train, indices_to_be_deleted)

In [ ]:
# indices = random.sample(range(len(X_train)), round(len(X_train)*0.5))

# X_flag = X_flag[indices]
# X_train = X_train[indices]
# y_train = y_train[indices]

In [ ]:
show_steering_angle_distribution(y_train)

positive_angle = [i for i in y_train if i > 0]
nagative_angle = [i for i in y_train if i < 0]
zero_angle = [i for i in y_train if i == 0]

print('number of positive angle:', len(positive_angle))
print('number of negative angle:', len(nagative_angle))
print('number of zero angle:', len(zero_angle))

# Saving

In [ ]:
out_file = open('driving_data.csv', 'w')
for i in range(len(X_train)):
    out_file.write(X_train[i] + ", " + str(X_flag[i]) + ", " + str(y_train[i]) + "\n")
out_file.close()